# Baseline

In [1]:
import numpy as np
import pandas as pd
# from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import math
import pickle
from sklearn.metrics import roc_auc_score
import optuna
import catboost as cb
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')
from tqdm.auto import tqdm

In [3]:
# !pip freeze | grep "numpy\|pandas\|lightgbm\|scikit-learn"

## Загрузка данных

In [2]:
train_df = pd.read_parquet("train_data.pqt")
test_df = pd.read_parquet("test_data.pqt")

заполянем 5 месяцем

In [3]:
test_df['start_cluster'] = test_df['start_cluster'].fillna(method='ffill')

вытягиваем в колбасу по id

In [4]:
grouped_df_first = test_df.groupby('id').first().reset_index()
merged_df = pd.merge(test_df, grouped_df_first, on='id', suffixes=('', '_first'))

grouped_df_second = test_df.groupby('id').nth(1).reset_index()
merged_df = pd.merge(merged_df, grouped_df_second, on='id', suffixes=('', '_second'))

merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

In [5]:
test_df = merged_df[merged_df['date'] == 'month_6']

In [6]:
grouped_df_first = train_df.groupby('id').first().reset_index()
merged_df = pd.merge(train_df, grouped_df_first, on='id', suffixes=('', '_first'))

grouped_df_second = train_df.groupby('id').nth(1).reset_index()
merged_df = pd.merge(merged_df, grouped_df_second, on='id', suffixes=('', '_second'))

merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

In [7]:
train_df = merged_df[merged_df['date'] == 'month_3']

удаляем сильную корреляцию

In [8]:
corr_matrix = train_df.corr().abs()

# Получение верхнего треугольника матрицы корреляции (без диагонали)
upper_triangle = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Нахождение колонок, где корреляция больше 0.9
to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.95)]

train_df =train_df.drop(to_drop, axis=1)


In [9]:
test_df =test_df.drop(to_drop, axis=1)

In [10]:
category_columns = ['start_cluster', 'channel_code', 'city', 'city_type',
                    'index_city_code', 'ogrn_month', 'ogrn_year', 'okved', 'segment']

In [11]:
# counts = train_df['city_type'].value_counts()

# # Построение гистограммы
# plt.bar(counts.index, counts.values)
# plt.xlabel('Уникальные значения')
# plt.ylabel('Частота')
# plt.title('Гистограмма количества встречающихся раз категориальных значений')
# plt.show()

обработка текстов и нан

In [12]:
train_df['channel_code']

2          channel_code_5
5          channel_code_2
8         channel_code_12
11        channel_code_14
14         channel_code_8
               ...       
599987     channel_code_9
599990    channel_code_14
599993     channel_code_8
599996     channel_code_9
599999    channel_code_14
Name: channel_code, Length: 200000, dtype: object

In [16]:
# def clever_one_hot(df, col): #переписать в lable encoding?
#     top_4 = df[col].value_counts().index[:4]
#     df.loc[~df[col].isin(top_4), col] = 'other'
#     one_hot_encoded = pd.get_dummies(df[col], prefix=col)
#     df = df.drop(col, axis=1)
#     return pd.concat([df, one_hot_encoded], axis=1)

def feature_prossesing(df):
    one_hot_encoded = pd.get_dummies(df['segment'], prefix='seg')
    df = df.drop('segment', axis=1)
    df = pd.concat([df, one_hot_encoded], axis=1)

    df['ogrn_year'] = df['ogrn_year'].fillna('_-1')
    df['ogrn_month'] = df['ogrn_month'].fillna('_-1')
    df['ogrn_year'] = df['ogrn_year'].apply(lambda x: int(x.split('_')[-1]))
    df['ogrn_month'] = df['ogrn_month'].apply(lambda x: int(x.split('_')[-1]))
    df['lasting'] = df['ogrn_year']*12 + df['ogrn_month']
    df['lasting'] = df['lasting'].apply(lambda x: x if x>=0 else -1)

    del df['ogrn_year']
    del df['ogrn_month']

    label_encoder = LabelEncoder()
    

    for i in ['channel_code', 'city_type', 'okved']:
        df[i] = label_encoder.fit_transform(df[i])

    
    df['start_cluster'] = label_encoder.fit_transform(df['start_cluster'])
    df['end_cluster'] = label_encoder.transform(df['end_cluster'])
    category_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

    df = df.drop(df.select_dtypes(include=['object']).columns, axis=1)
    df = df.fillna(-1)

    return df, category_mapping   

In [17]:
df, category_mapping = feature_prossesing(train_df)

In [18]:
def clever_one_hot(df, col, df1): #переписать в lable encoding?
    top_4 = df1[col].value_counts().index[:4]
    df.loc[~df[col].isin(top_4), col] = 'other'
    one_hot_encoded = pd.get_dummies(df[col], prefix=col)
    df = df.drop(col, axis=1)
    return pd.concat([df, one_hot_encoded], axis=1)

def feature_prossesing(df, df_1):
    one_hot_encoded = pd.get_dummies(df['segment'], prefix='seg')
    df = df.drop('segment', axis=1)
    df = pd.concat([df, one_hot_encoded], axis=1)

    df['ogrn_year'] = df['ogrn_year'].fillna('_-1')
    df['ogrn_month'] = df['ogrn_month'].fillna('_-1')
    df['ogrn_year'] = df['ogrn_year'].apply(lambda x: int(x.split('_')[-1]))
    df['ogrn_month'] = df['ogrn_month'].apply(lambda x: int(x.split('_')[-1]))
    df['lasting'] = df['ogrn_year']*12 + df['ogrn_month']
    df['lasting'] = df['lasting'].apply(lambda x: x if x>=0 else -1)

    del df['ogrn_year']
    del df['ogrn_month']

    label_encoder = LabelEncoder()
    

    for i in ['channel_code', 'city_type', 'okved']:
        df[i] = label_encoder.fit_transform(df[i])

    df['start_cluster'] = label_encoder.fit_transform(df['start_cluster'])
    #df['end_cluster'] = label_encoder.transform(df['end_cluster'])
    category_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

    df = df.drop(df.select_dtypes(include=['object']).columns, axis=1)
    df = df.fillna(-1)

    return df, category_mapping   

In [19]:
test, category_mapping = feature_prossesing(test_df, train_df)

In [27]:
category_mapping

{'{other}': 0,
 '{}': 1,
 '{α, β}': 2,
 '{α, γ}': 3,
 '{α, δ}': 4,
 '{α, ε, η}': 5,
 '{α, ε, θ}': 6,
 '{α, ε, ψ}': 7,
 '{α, ε}': 8,
 '{α, η}': 9,
 '{α, θ}': 10,
 '{α, λ}': 11,
 '{α, μ}': 12,
 '{α, π}': 13,
 '{α, ψ}': 14,
 '{α}': 15,
 '{λ}': 16}

веса для рок аука которые идут в бустинг

In [20]:
def weighted_roc_auc(y_true, y_pred, labels, weights_dict):
    unnorm_weights = np.array([weights_dict[label] for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()
    classes_roc_auc = roc_auc_score(y_true, y_pred, labels=labels,
                                    multi_class="ovr", average=None)
    return sum(weights * classes_roc_auc)

In [22]:
cluster_weights = pd.read_excel("cluster_weights.xlsx").set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()

In [23]:
weits = [0]*17

for k, v in category_mapping.items():
    weits[category_mapping[k]] = weights_dict[k]

In [24]:
weits

[3, 1, 3, 3, 3, 1, 1, 3, 2, 2, 1, 3, 2, 1, 3, 2, 2]

треним

In [25]:
y = df['end_cluster']
X = df.drop(['end_cluster'], axis=1)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [30]:
# !pip install lightautoml

In [27]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import torch

# LightAutoML presets, task and report generation
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
from lightautoml.report.report_deco import ReportDeco

In [28]:
task = Task('multiclass')

In [83]:
automl = TabularUtilizedAutoML(
    task = Task('multiclass'),
    timeout = 3600,
)

In [84]:
oof_pred = automl.fit_predict(
    df,
    roles = {'target': 'end_cluster'}
    , verbose = 1
)

[02:43:03] Start automl utilizator with listed constraints:
[02:43:03] - time: 3600.00 seconds
[02:43:03] - CPU: 4 cores
[02:43:03] - memory: 16 GB

[02:43:03] If one preset completes earlier, next preset configuration will be started

[02:43:03] ==================================================
[02:43:03] Start 0 automl preset configuration:
[02:43:03] d:\Anaconda\lib\site-packages\lightautoml\automl\presets\tabular_configs\conf_0_sel_type_0.yml, random state: {'reader_params': {'random_state': 42}, 'nn_params': {'random_state': 42}, 'general_params': {'return_all_predictions': False}}
[02:43:03] Stdout logging level is INFO.
[02:43:03] Task: multiclass

[02:43:03] Start automl preset with listed constraints:
[02:43:03] - time: 3600.00 seconds
[02:43:03] - CPU: 4 cores
[02:43:03] - memory: 16 GB

[02:43:03] Train data shape: (200000, 120)

[02:44:59] Layer 1 train process start. Time left 3483.39 secs
[02:45:09] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[02:50:31] Time limit exce

In [74]:
ans = automl.predict(test)

In [81]:
type(ans.data)

numpy.ndarray

In [82]:
data = {} #{'id': test.id.to_list()}
for cls, prob in zip(category_mapping.keys(), ans):
    data[cls] = prob

column_mapping = {
    0: '{other}',
    1: '{}',
    2: '{α, β}',
    3: '{α, γ}',
    4: '{α, δ}',
    5: '{α, ε, η}',
    6: '{α, ε, θ}',
    7: '{α, ε, ψ}',
    8: '{α, ε}',
    9: '{α, η}',
    10: '{α, θ}',
    11: '{α, λ}',
    12: '{α, μ}',
    13: '{α, π}',
    14: '{α, ψ}',
    15: '{α}',
    16: '{λ}'
}

output = pd.DataFrame(ans.data)
output = output.rename(columns=column_mapping)
output = output.assign(id=test['id'].tolist())

sample_submission_df = pd.read_csv("sample_submission.csv")
output[list(sample_submission_df.columns)].to_csv('ans100.csv', index=False)


In [ ]:
import pandas as pd
sample_submission_df = pd.read_csv("sample_submission.csv")
list(sample_submission_df.columns)

['id',
 '{other}',
 '{}',
 '{α, β}',
 '{α, γ}',
 '{α, δ}',
 '{α, ε, η}',
 '{α, ε, θ}',
 '{α, ε, ψ}',
 '{α, ε}',
 '{α, η}',
 '{α, θ}',
 '{α, λ}',
 '{α, μ}',
 '{α, π}',
 '{α, ψ}',
 '{α}',
 '{λ}']

In [ ]:
import hashlib

def generate_md5(file_path):
    # Open the file in binary mode
    with open(file_path, "rb") as file:
        # Read the content of the file
        content = file.read()
        # Generate the MD5 hash of the content
        md5_hash = hashlib.md5(content).hexdigest()
        return md5_hash

# Call the function to generate the MD5 hash of a video file
# Replace "video.mp4" with the path to your video file
md5_hash = generate_md5(r"C:\Users\alina\Videos\2024-03-12 14-31-10.mkv")

# Print the MD5 hash
print(md5_hash)


e4ead55ff0a49c8ee3cc879f2470f4d9
